In [1]:
import os
import random
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [2]:
os.environ['PYSPARK_PYTHON'] = 'python3'

In [3]:
spark = SparkSession.builder.appName('PySpark').getOrCreate()

In [4]:
schema = StructType(
    [
        StructField('region',              StringType(), False), 
        StructField('tik',                 StringType(), False), 
        StructField('uik',                 StringType(), False), 
        StructField('voters_number',       IntegerType(), False),
        StructField('bulletins_number',    IntegerType(), False),
        StructField('early_bulletins',     IntegerType(), False),
        StructField('indoors_bulletins',   IntegerType(), False),
        StructField('outdoors_bulletins',  IntegerType(), False),
        StructField('canceled_bulletins',  IntegerType(), False),
        StructField('outbox_bulletins',    IntegerType(), False),
        StructField('inbox_bulletins',     IntegerType(), False),
        StructField('invalid_bulletins',   IntegerType(), False),
        StructField('valid_bulletins',     IntegerType(), False),
        StructField('lost_bulletins',      IntegerType(), False),
        StructField('uncounted_bulletins', IntegerType(), False),
        StructField('baburin',             IntegerType(), False),
        StructField('grudinin',            IntegerType(), False),
        StructField('zhirinovsky',         IntegerType(), False),
        StructField('putin',               IntegerType(), False),
        StructField('sobchak',             IntegerType(), False),
        StructField('suraykin',            IntegerType(), False),
        StructField('titov',               IntegerType(), False),
        StructField('yavlinsky',           IntegerType(), False)
    ]
)

In [5]:
df = spark.read.csv('export/elections.csv', schema = schema)

In [6]:
df.createOrReplaceTempView('elections')

In [7]:
turnout = spark.sql(
    '''SELECT region, (SUM(invalid_bulletins) + SUM(valid_bulletins)) / SUM(voters_number) * 100 as turnout 
    FROM elections
    GROUP BY region
    ORDER BY turnout DESC'''
)

turnout.show()

+--------------------+-----------------+
|              region|          turnout|
+--------------------+-----------------+
|Территория за пре...|98.01821236184206|
|     Республика Тыва|93.62542974951742|
|Ямало-Ненецкий ав...|91.87213306617981|
|Кабардино-Балкарс...| 91.6978754085207|
|Чеченская Республика|91.50929703298173|
|Республика Северн...|89.93704831730909|
| Республика Дагестан|87.44351419820495|
|Карачаево-Черкесс...|  87.343809390099|
| Кемеровская область|83.07556958224816|
|Чукотский автоном...|82.26051697921946|
|Республика Ингушетия|81.95937165135314|
|    Брянская область|79.65976807571519|
|   Тюменская область| 78.8829673763749|
| Республика Мордовия|77.83648793754097|
|  Краснодарский край|77.83466166202626|
|Республика Татарс...| 77.3652654820273|
|Чувашская Республ...|76.21061419670117|
|Республика Башкор...|75.42120065745192|
|  Республика Бурятия|75.19133382510174|
|Республика Адыгея...| 74.3059515324305|
+--------------------+-----------------+
only showing top

In [8]:
candidates = ['baburin', 'grudinin', 'zhirinovsky', 'putin', 'sobchak', 'suraykin', 'titov', 'yavlinsky']

In [9]:
random_candidate = random.choice(candidates)

In [10]:
random_candidate

'putin'

In [11]:
random_candidate_best = spark.sql(
    f'''SELECT region, tik, uik, {random_candidate} / voters_number * 100 as {random_candidate}_percent
    FROM elections
    WHERE invalid_bulletins + valid_bulletins > 300
    ORDER BY {random_candidate}_percent DESC'''
)
random_candidate_best.show()

+--------------------+--------------------+----+-----------------+
|              region|                 tik| uik|    putin_percent|
+--------------------+--------------------+----+-----------------+
|     Республика Тыва|           Эрзинская| 180|            100.0|
|     Республика Тыва|      Бай-Тайгинская|  43|            100.0|
| Кемеровская область|Прокопьевск, Цент...| 876|            100.0|
|     Республика Тыва|        Улуг-Хемская| 159|            100.0|
|Республика Татарс...|         Апастовская| 909|99.76415094339622|
|     Республика Тыва|         Тес-Хемская| 145|99.73045822102425|
|     Республика Тыва|      Бай-Тайгинская|  45|99.72714870395635|
|     Республика Тыва|        Улуг-Хемская| 165| 99.6594778660613|
|     Республика Тыва|        Улуг-Хемская| 163|99.63325183374083|
|Республика Башкор...|    Уфа, Калининская|  98|99.56331877729258|
|Ямало-Ненецкий ав...|           Ямальская|1310|99.51590594744121|
|Ямало-Ненецкий ав...|           Ямальская|1311|  99.511684687

In [12]:
turnout_max_difference_region = spark.sql(
    '''SELECT region, MAX(turnout) - MIN(turnout) as max_min_difference
    FROM
        (SELECT region, tik, (SUM(invalid_bulletins) + SUM(valid_bulletins)) / SUM(voters_number) * 100 as turnout
        FROM elections
        GROUP BY region, tik)
    GROUP BY region
    ORDER BY max_min_difference DESC
    LIMIT 1'''
)

turnout_max_difference_region.show()

+--------------------+------------------+
|              region|max_min_difference|
+--------------------+------------------+
|Архангельская обл...|  49.8591648406849|
+--------------------+------------------+



In [13]:
variance = spark.sql(
    '''SELECT
        region,
        VARIANCE((invalid_bulletins + valid_bulletins) / voters_number) as variance,
        STDDEV((invalid_bulletins + valid_bulletins) / voters_number) as standard_deviation
    FROM elections
    GROUP BY region'''
)

variance.show()

+--------------------+--------------------+-------------------+
|              region|            variance| standard_deviation|
+--------------------+--------------------+-------------------+
|Республика Саха (...|0.012938672389188005|0.11374828521427478|
|Калининградская о...|0.014347099725678693|0.11977937938426085|
|Новосибирская обл...|0.009977412359718732|0.09988699795127859|
|   город Севастополь|0.004586064984736387|0.06772049161617469|
|Свердловская область|0.007134732640751302|0.08446734659471257|
|  Республика Хакасия|0.013749163498496995|0.11725682708694192|
| Магаданская область|0.020700450180778953|0.14387651017723133|
|Республика Ингушетия|0.004868517898215...| 0.0697747654830594|
|Удмуртская Респуб...|0.009330475533570205|0.09659438665662827|
|     Камчатский край|0.028471418481428836|0.16873475777512123|
| Саратовская область|0.023797786693646066|0.15426531267153373|
|Республика Башкор...|0.015107853188060046|0.12291400728989371|
|  Ростовская область|0.0172624157153348

In [14]:
for candidate in candidates:
    candidate_results_by_percent = spark.sql(
        f'''SELECT {candidate}_result, COUNT({candidate}_result) as uiks_number FROM
                (SELECT uik, int(100 * {candidate} / voters_number) as {candidate}_result
                FROM elections)
            GROUP BY {candidate}_result
            ORDER BY {candidate}_result'''
    )

    candidate_results_by_percent.show(101)

+--------------+-----------+
|baburin_result|uiks_number|
+--------------+-----------+
|             0|      92374|
|             1|       4471|
|             2|        522|
|             3|        153|
|             4|         70|
|             5|         30|
|             6|         21|
|             7|         17|
|             8|          8|
|             9|          5|
|            10|          8|
|            11|          2|
|            12|          1|
|            13|          1|
|            14|          3|
|            16|          2|
|            17|          1|
|            18|          1|
|            20|          2|
|            21|          1|
|            23|          1|
|            44|          1|
+--------------+-----------+

+---------------+-----------+
|grudinin_result|uiks_number|
+---------------+-----------+
|              0|       2472|
|              1|       3745|
|              2|       4358|
|              3|       5269|
|              4|       6972|
|    

+---------------+-----------+
|suraykin_result|uiks_number|
+---------------+-----------+
|              0|      91174|
|              1|       5128|
|              2|        795|
|              3|        226|
|              4|        142|
|              5|         63|
|              6|         52|
|              7|         47|
|              8|         44|
|              9|         10|
|             10|          8|
|             11|          1|
|             12|          1|
|             13|          1|
|             14|          1|
|             16|          2|
+---------------+-----------+

+------------+-----------+
|titov_result|uiks_number|
+------------+-----------+
|           0|      90179|
|           1|       6733|
|           2|        475|
|           3|        140|
|           4|         66|
|           5|         34|
|           6|         19|
|           7|         13|
|           8|         10|
|           9|          4|
|          10|          4|
|          11|       